## Notebook for Reproducing Genetic Algrithm Results

### import packages and other code components

In [1]:
import networkx as nx
from pgmpy.models import BayesianNetwork
import matplotlib.pyplot as plt
from pgmpy.estimators import K2Score,BDsScore
import pandas as pd
import tools as tools
import DisplayGraph as dg
import ReadCSV as rcsv
import pgmpy
import timeit
import copy    
import pickle
import GeneticAlgorithm as GA
import TwoDMatrixIterator as TMI
import BinaryStringToBayesianN as BSTB
import networkx as nx

### define utility functions

In [2]:


def FindLastColon( string ):
    size = len( string )
    for i in range( size ):
        if ( string[ size -1 -i ] == ":"):
            return size-1-i
    return 0

# construct a graph according to the significant edges found by the permutation test
def SignificantEdgesToGraph( filename ):
    g = nx.DiGraph()
    file1 = open( filename, 'r')
    Lines = file1.readlines()
    edges = []
    for line in Lines:
        bounds = GetVerticesFromString( line )
        g.add_edge( bounds[0], bounds[1] )
        edges.append( (bounds[0], bounds[1]) )
    file1.close()
    return g

# get node names
def GetVerticesFromString( string ):
    pre_strings = string.split( ";;;" )
    strings = pre_strings[0].split( "--- ")
    last_colon = FindLastColon( strings[1] )
    strings[1] = strings[ 1 ][ 0:last_colon ]
    return strings

# 
def InitializeVarToIndexDictionary( IndexToVarListe ):
    var_to_index = {}
    for i in range( len( IndexToVarListe ) ):
        var_to_index[ IndexToVarListe[i] ] = i
    return var_to_index


def btog(b,es,g1):
    # binary representation to graph
    # b: binary string
    # es: list of bidirectional edges
    #     each element is a list of 2 edges
    #     each edge is a tuple of 2 nodes
    #     (list of lists of tuples)
    # g: the intial graph with bidirectional edges
    
    to_rm = []
    g2=copy.deepcopy(g1)
    for j in range(13):  # remove 13 out of 26 edges
        if b[j] == '0':
            to_rm.append(es[j][0])
        else:
            to_rm.append(es[j][1])
    for e in to_rm:
        g2.remove_edge(e[0],e[1])
    return g2

# write a txt containing bidirectional edges
def bidirectional_edges():
    f= open('e.txt')
    lines = f.readlines()
    bie=[]
    for l in lines:
        #print(l)
        n1=l.split('---')[0]
        n2=l.split('---')[1][:-1]
        bie.append((n1,n2))
    
    es=[]
    for i in range(13):
        es.append([bie[2*i],bie[2*i+1]])
    return es

# output file for visualizing the graph in Cytoscape
def ToCytoscape( matrix, index_to_vertex , name = 'aaaaa.csv' ):
    f = open( name, "w" )
    f.write( ",Left,Right\n" )
    cnt=0
    for i in range( len( matrix ) ):
        for j in range( len( matrix[i] ) ):
            if ( matrix[i][j] == 1 ):
                v1 = index_to_vertex[i]
                v2 = index_to_vertex[j]
                f.write( str(cnt) + "," +  v1 + "," + v2 )
                f.write( "\n" )
                cnt+=1
    f.close()


### MAIN

In [ ]:
# build a grpah according to the significant edges found by the permutation test
g = SignificantEdgesToGraph( "notcorrectedSorted11005.txt")

# get node names
index_to_var_liste = list( g.nodes )
var_to_index_dict = InitializeVarToIndexDictionary( index_to_var_liste )

# load the dataset for scoring the graph
data=pd.read_csv('ScoringDataframe.csv')

    
# get bidirectional edges
es = bidirectional_edges()
        
#build a dictionary recording the scores for all network candidates
#keys are network binary representations
#values are total K2 or BDs scores

#the candidates are generated by choosing one direction from the bidirectional edges 

#TAKES ABOUT 1H20MIN TO RUN
#NO NEED TO RUN IF SCORING HAS BEEN FINISHED 
# IN THAT CASE, SIMPLY LOAD THE DICTIONARY (SKIP THIS PART)
'''
scoredic={}
for i in range(2**13):
    representation = format(i, '#015b')[2:]
    to_rm = []
    g2=copy.deepcopy(g)
    for j in range(13):
        if representation[j] == '0':
            to_rm.append(es[j][0])
        else:
            to_rm.append(es[j][1])
    for e in to_rm:
        g2.remove_edge(e[0],e[1])
    graphscore = ScoreGraph(g2,'bd',data)
    scoredic[representation]=graphscore
        
pickle.dump(scoredic,open('networkBDscores.pickle','wb'))    
'''

    
# load a dictionary recording the scores for all network candidates
scoredic = pickle.load(open('networkscores.pickle','rb'))
    
    
'''select top k candidates'''
k=10
firstnet = format(0, '#015b')[2:]
firstscore = scoredic[firstnet]
topk_b=[]
topk_scores=[]
for i in range(k):
    topk_b.append(firstnet)
    topk_scores.append(firstscore)
for representation in scoredic.keys():
    score = scoredic[representation]
    for i in range(k):
        if (topk_scores[i] < score):
            topk_scores[i] = score
            topk_b[i] = representation
            break
    
    
''' get topk graphs and adjacency matrices'''
topk_g=[]
adjs=[]
for b in topk_b:
    candidate = btog(b,es,g)
    if nx.is_directed_acyclic_graph(candidate)==False:
        print('not DAG!')
    topk_g.append(candidate)
    adj = BSTB.NetworkTo2dMatrix( candidate, var_to_index_dict )
    adjs.append(adj)
    
    
# run the genetic algorithm
#start_parents, end_thresh, mutate_numb, best_cand_num, bad_reprod_accept, mutate_perc, index_to_vertex 
children = GA.GeneticRun( adjs, 0.0001, 5, 10, 2, 0.05, index_to_var_liste )
    
bestDescendant = children.list[0]
graph = BSTB.MatrixToNetwork( bestDescendant, index_to_var_liste )
    
ug = graph.to_undirected()

ToCytoscape( bestDescendant, index_to_var_liste, name = 'final_structure.csv' )  # name = "Final_structure.csv"

-359975.1721076082
duration 130.46846234612167
-359434.41332173184
duration 165.26137952925637
-359434.41332173184
duration 181.08281159494072
-359434.41332173184
